In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU , Bidirectional, Dropout
from keras.callbacks import ModelCheckpoint
from random import randint
import tensorflow as tf

In [2]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('abba.csv')

# Concatenate all lyrics into a single string
corpus = ' '.join(data['lyrics'].dropna())

# Print the first 500 characters of the corpus just to verify
print(corpus[:500])

My my
At Waterloo Napoleon did surrender
Oh yeah
And I have met my destiny in quite a similar way
The history book on the shelf
Is always repeating itself
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
My my
I tried to hold you back, but you were stronger
Oh yeah
And now it seems my only chance is giving up the fight
And how co


In [3]:
import re

# Function to clean the text
def clean_text(text):
    # Remove Unicode characters
    cleaned_text = re.sub('[^\x00-\x7F]+', '', text)
    # Remove \r and other special characters
    cleaned_text = re.sub(r'[\r\n\t]', ' ', cleaned_text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Clean the corpus
corpus = clean_text(corpus)

# Save the cleaned corpus to a file
with open('cleaned_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(corpus)

print("Data cleaned and saved to 'cleaned_corpus.txt'.")

Data cleaned and saved to 'cleaned_corpus.txt'.


In [4]:
# Get unique characters from the corpus
unique_chars = sorted(set(corpus))

# Create encoder dictionary (character to integer)
encoder_dict = {char: i for i, char in enumerate(unique_chars)}

# Create decoder dictionary (integer to character)
decoder_dict = {i: char for i, char in enumerate(unique_chars)}

# Print encoder dictionary
print("Encoder dictionary:")
print(encoder_dict)

# Print decoder dictionary
print("\nDecoder dictionary:")
print(decoder_dict)


Encoder dictionary:
{' ': 0, '!': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '6': 15, ':': 16, '?': 17, 'A': 18, 'B': 19, 'C': 20, 'D': 21, 'E': 22, 'F': 23, 'G': 24, 'H': 25, 'I': 26, 'J': 27, 'K': 28, 'L': 29, 'M': 30, 'N': 31, 'O': 32, 'P': 33, 'Q': 34, 'R': 35, 'S': 36, 'T': 37, 'U': 38, 'V': 39, 'W': 40, 'Y': 41, 'Z': 42, '[': 43, ']': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, '\x7f': 71}

Decoder dictionary:
{0: ' ', 1: '!', 2: '"', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '/', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '6', 16: ':', 17: '?', 18: 'A', 19: 'B', 20: 'C', 21: 'D', 22: 'E', 23: 'F', 24: 'G', 25: 'H', 26: 'I', 27: 'J', 28: 'K', 29: 'L', 30: 'M', 31: 'N', 32: 'O', 33: 'P', 34: 'Q', 35: 'R', 36

In [5]:
# slice the corpus into semi-redundant sequences of 20 characters, and encode them using our dictionaries.
# Print the first 20 characters of the corpus just to verify
# sliced_corpus = cleaned_corpus[:20]
# encoder_dict(sliced_corpus)

sentence_length = 20

skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoder_dict[char] for char in sentence])
    y_data.append(encoder_dict[next_char])

In [6]:
# simple check.
X_data[1], y_data[1]


([69, 0, 57, 69, 0, 18, 64, 0, 40, 45, 64, 49, 62, 56, 59, 59, 0, 31, 45, 60],
 59)

In [7]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

Sliced our corpus into 210090 sentences of length 20


In [8]:
# now we need one-hot encoding
num_chars = len(unique_chars)

print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=bool)
y = np.zeros((num_sentences, num_chars), dtype=bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [9]:
# Define our model
print("Let's build model.")
model = Sequential()
model.add(LSTM(32, input_shape=(sentence_length, num_chars), return_sequences=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

Let's build model.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 32)            13440     
                                                                 
 bidirectional (Bidirection  (None, 20, 64)            16640     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 20, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 72)                4680      
                                   

In [10]:
# Train our model! save the weights after each epoch if it's a new best for loss and save model architecture.
architecture = model.to_json()
with open('model.json', 'w') as model_file:
    model_file.write(architecture)

# Set up checkpoints, and save trained model
file_path="weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]



In [11]:
# Train het model voor minimaal 20 epochs, En selecteer het dat weights-file die de beste resultaten geeft?
# Is er ook een weights-file waarvanje kunt zeggen dat er overfitting optreedt?

# kijk goed naar de onderstaande regel.

history = model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)